# LLM Fundamentals with LangChain

## Using LLMs in LangChain

In [2]:
# LLMs

from langchain_ollama.llms import OllamaLLM

model = OllamaLLM(model='llama3.1')

model.invoke("The sky is")

'...blue! (or maybe grey, or cloudy, or sunny...) What were you going to say?'

In [3]:
# Chat Model

from langchain_ollama.chat_models import ChatOllama
from langchain_core.messages import HumanMessage

model = ChatOllama(model="llama3.1")
prompt = [HumanMessage("What is the capital of France?")]

model.invoke(prompt)

AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-10-24T14:31:15.3324601Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1180698400, 'load_duration': 109208800, 'prompt_eval_count': 17, 'prompt_eval_duration': 413545100, 'eval_count': 8, 'eval_duration': 650112000, 'model_name': 'llama3.1', 'model_provider': 'ollama'}, id='lc_run--85082919-006a-4d56-a4c4-baf469dcd87f-0', usage_metadata={'input_tokens': 17, 'output_tokens': 8, 'total_tokens': 25})

In [4]:
# Chat Model with SystemMessage interaction

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ollama.chat_models import ChatOllama

model = ChatOllama(model='llama3.1')
system_msg = SystemMessage(
    '''You are a helpful assistant that responds to questions with three exclamation marks.'''
)
human_msg = HumanMessage(
    '''What is the capital of France?'''
)

model.invoke([system_msg, human_msg])

AIMessage(content='Paris!!', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-10-24T14:31:16.8282216Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1469849700, 'load_duration': 93859400, 'prompt_eval_count': 37, 'prompt_eval_duration': 1172635500, 'eval_count': 3, 'eval_duration': 183269000, 'model_name': 'llama3.1', 'model_provider': 'ollama'}, id='lc_run--3eb70514-d595-405b-bc97-e1c83da684fe-0', usage_metadata={'input_tokens': 37, 'output_tokens': 3, 'total_tokens': 40})

## Making LLM Prompts Reusable

In [5]:
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import PromptTemplate

# both 'template' and 'model' can be reused many times

template = PromptTemplate.from_template(
    '''Answer the question based on the context below. 
    If the question cannot be answered using the information provided, answer "I don't know."

    Context: {context}
    
    Question: {question}

    Answer:'''
)

model = OllamaLLM(model='llama3.1')

# 'prompt' and 'completion' are the results of using template and model once

prompt = template.invoke({
    "context": """The most recent advancements in NLP are being driven by Large Language Models (LLMs). These models outperforms their
        smaller counterparts and have become invaluable for developers who are creating applications with NLP capabilities. Developers
        can tap into tese models through Hugging Face's 'transformers' library, or by utilizing OpenAI and Cohere's offerings through the
        'openai' and 'cohere' libraries, respectively.""",
    "question":"Which model providers offer LLMs?"
})

completion = model.invoke(prompt)
completion

"OpenAI and Cohere offer LLMs through their respective libraries ('openai' and 'cohere')."

In [7]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

# both 'template' and 'model' can be reused many times

template = ChatPromptTemplate.from_messages([
    ('system', '''Answer the question basen on the context below. If the question cannot be answered using hte information provided,
        answer with "I don't know".'''),
    ('human', 'Context: {context}'),
    ('human', 'Question: {question}')
])

model = ChatOllama(model='llama3.1')

# 'prompt' and 'completion' are the results of using template and model once

prompt = template.invoke({
    "context": """The most recent advancements in NLP are being driven by Large Language Models (LLMs). These models outperforms their
        smaller counterparts and have become invaluable for developers who are creating applications with NLP capabilities. Developers
        can tap into tese models through Hugging Face's 'transformers' library, or by utilizing OpenAI and Cohere's offerings through the
        'openai' and 'cohere' libraries, respectively.""",
    "question":"Which model providers offer LLMs?"
})

model.invoke(prompt)

AIMessage(content="Hugging Face ('transformers' library), OpenAI ('openai' library), and Cohere ('cohere' library) offer Large Language Models (LLMs).", additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-10-24T14:41:04.4695804Z', 'done': True, 'done_reason': 'stop', 'total_duration': 15346473400, 'load_duration': 6982090000, 'prompt_eval_count': 148, 'prompt_eval_duration': 5250743800, 'eval_count': 35, 'eval_duration': 3084087000, 'model_name': 'llama3.1', 'model_provider': 'ollama'}, id='lc_run--b4019ef5-6dd2-4b55-8640-634b0bee301b-0', usage_metadata={'input_tokens': 148, 'output_tokens': 35, 'total_tokens': 183})

## Getting Specific Formats out of LLMs

### JSON Output

In [3]:
from langchain_ollama import ChatOllama
from pydantic import BaseModel

class AnswerWithJustification(BaseModel):
    """An answer to the user's question along with justification for the answer."""
    answer: str
    """The answer to the user's question"""
    justification: str
    """Justification for the answer"""

llm = ChatOllama(model='llama3.1', temperature=0)
structured_llm = llm.with_structured_output(AnswerWithJustification)

structured_llm.invoke("What weighs more, a pound of bricks or a pound of feathers")

AnswerWithJustification(answer='They weigh the same!', justification="One pound is a unit of weight, so it's equal to one pound regardless of what you're measuring. The difference is in their density and volume.")

### Other Machine-Readable Formats with Output Parsers

In [6]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()
items = parser.invoke("apple, banana, cherry")

items

['apple', 'banana', 'cherry']

## Assembling the Many Pieces of an LLM Application

### Using the Runnable Interface

In [8]:
from langchain_ollama import ChatOllama

model = ChatOllama(model='llama3.1')

completion = model.invoke('Hi there!')
print(completion)

completions = model.batch(['Hi there!', 'Bye!'])
print(completions)

for token in model.stream('Bye!'):
    print(token)

content="How's it going? Is there something I can help you with or would you like to chat?" additional_kwargs={} response_metadata={'model': 'llama3.1', 'created_at': '2025-10-27T09:28:29.9338158Z', 'done': True, 'done_reason': 'stop', 'total_duration': 11136677900, 'load_duration': 8536520000, 'prompt_eval_count': 13, 'prompt_eval_duration': 636706900, 'eval_count': 21, 'eval_duration': 1934901600, 'model_name': 'llama3.1', 'model_provider': 'ollama'} id='lc_run--28688c97-569b-4c7c-ac17-13eb8df58de1-0' usage_metadata={'input_tokens': 13, 'output_tokens': 21, 'total_tokens': 34}
[AIMessage(content="How's it going? Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-10-27T09:28:31.9972715Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2044071800, 'load_duration': 135746200, 'prompt_eval_count': 13, 'prompt_eval_duration': 99911100, 'eval_count': 21, 'eval_duration': 1795024100, '

### Imperative Composition

In [9]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain

# the building blocks

template = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant.'),
    ('human', '{question}')
])

model = ChatOllama(model='llama3.1')

# combine them in a function
# @chain decorator adds the same Runnable interface for any function you write

@chain
def chatbot(values):
    prompt = template.invoke(values)
    return model.invoke(prompt)

# use it

chatbot.invoke({'question': 'Which model providers offer LLMs'})

AIMessage(content='There are several model providers that offer Large Language Models (LLMs). Here are some of the most popular ones:\n\n1. **Hugging Face**: Hugging Face offers a range of pre-trained LLMs, including BERT, RoBERTa, and Longformer, through their Transformers library.\n2. **Google Cloud AI Platform**: Google Cloud AI Platform provides access to pre-trained LLMs like BERT, RoBERTa, and XLNet, which can be fine-tuned for specific tasks.\n3. **Microsoft Azure Machine Learning**: Microsoft Azure Machine Learning offers a range of pre-trained LLMs, including BERT and RoBERTa, which can be used for natural language processing (NLP) tasks.\n4. **Amazon SageMaker**: Amazon SageMaker provides access to pre-trained LLMs like BERT and RoBERTa, which can be fine-tuned for specific NLP tasks.\n5. **DeepMind**: DeepMind offers a range of pre-trained LLMs, including Transformer-XL and XLNet, through their TensorFlow framework.\n6. **Meta AI**: Meta AI provides access to pre-trained LLM

In [10]:
# Streaming support can be also added

from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain

# the building blocks

template = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant.'),
    ('human', '{question}')
])

model = ChatOllama(model='llama3.1')

# combine them in a function
# @chain decorator adds the same Runnable interface for any function you write

@chain
def chatbot(values):
    prompt = template.invoke(values)
    for token in model.stream(prompt):
        yield token

# use it

for part in chatbot.stream({
    'question': 'Which model providers offer LLMs?'
}):
    print(part)

content='Several' additional_kwargs={} response_metadata={} id='lc_run--f5fbb422-088f-4dc1-a246-50d3ddfe09ba'
content=' models' additional_kwargs={} response_metadata={} id='lc_run--f5fbb422-088f-4dc1-a246-50d3ddfe09ba'
content=' provide' additional_kwargs={} response_metadata={} id='lc_run--f5fbb422-088f-4dc1-a246-50d3ddfe09ba'
content=' Large' additional_kwargs={} response_metadata={} id='lc_run--f5fbb422-088f-4dc1-a246-50d3ddfe09ba'
content=' Language' additional_kwargs={} response_metadata={} id='lc_run--f5fbb422-088f-4dc1-a246-50d3ddfe09ba'
content=' Models' additional_kwargs={} response_metadata={} id='lc_run--f5fbb422-088f-4dc1-a246-50d3ddfe09ba'
content=' (' additional_kwargs={} response_metadata={} id='lc_run--f5fbb422-088f-4dc1-a246-50d3ddfe09ba'
content='LL' additional_kwargs={} response_metadata={} id='lc_run--f5fbb422-088f-4dc1-a246-50d3ddfe09ba'
content='Ms' additional_kwargs={} response_metadata={} id='lc_run--f5fbb422-088f-4dc1-a246-50d3ddfe09ba'
content=')' additional_

In [11]:
# For asynchronous execution

from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain

# the building blocks

template = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant.'),
    ('human', '{question}')
])

model = ChatOllama(model='llama3.1')

# combine them in a function
# @chain decorator adds the same Runnable interface for any function you write

@chain
async def chatbot(values):
    prompt = await template.ainvoke(values)
    return await model.ainvoke(prompt)

# use it

await chatbot.ainvoke({
    'question': 'Which model providers offer LLMs?'
})

AIMessage(content='Several model providers offer Large Language Models (LLMs). Here are some of the most notable ones:\n\n1. **Hugging Face Transformers**: Offers a wide range of pre-trained models, including BERT, RoBERTa, DistilBERT, and more.\n2. **DeepSpeed**: Provides large language models, such as Megatron-LM and SwitchTransformers, optimized for high-performance computing.\n3. **Google Cloud AI Platform**: Offers pre-trained LLMs like BERT and RoBERTa, as well as the ability to fine-tune custom models.\n4. **Amazon SageMaker**: Provides access to pre-trained LLMs like BERT and DistilBERT, as well as tools for building and deploying custom models.\n5. **Microsoft Azure Machine Learning**: Offers a range of pre-trained LLMs, including BERT and RoBERTa, as well as tools for fine-tuning custom models.\n6. **IBM Watson Studio**: Provides access to pre-trained LLMs like BERT and DistilBERT, as well as tools for building and deploying custom models.\n7. **NVIDIA NGC**: Offers a range o

### Declarative Composition

In [1]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

# the building block

template = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant.'),
    ('human', '{question}')
])

model = ChatOllama(model='llama3.1')

# combine them with the | operator

chatbot = template | model

# use it

chatbot.invoke({'question': 'Which model providers offer LLMs?'})

AIMessage(content='Several companies and organizations provide Large Language Models (LLMs). Some notable ones include:\n\n1. **Meta AI**: Offers a range of pre-trained models, including the popular Transformers-based models like BART, T5, and DistilBERT.\n2. **Hugging Face**: Provides a suite of LLMs through their Transformers library, including models like RoFormer, Longformer, and XLNet.\n3. **Google Cloud AI Platform**: Offers various LLMs, such as BERT, RoBERTa, and ALBERT, for use in Google Cloud services.\n4. **IBM Watson Studio**: Provides access to a range of LLMs, including the IBM Watson Natural Language Understanding model.\n5. **Microsoft Azure Cognitive Services**: Offers several LLM-based APIs, like Text Analytics and Natural Language Processing (NLP).\n6. **Amazon SageMaker**: Supports various LLMs, including BERT, RoBERTa, and XLNet, through their pre-built algorithms.\n\nKeep in mind that this is not an exhaustive list, as new models are being developed and released r